In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py

Using TensorFlow backend.


In [5]:
# 提取特征向量方法；该方法可以使用选定的模型对数据进行处理，导出样本图像的特征向量。这部工作需要较大计算量，在本项目中是在aws上完成的。
# MODEL：模型名称, image_size：输入数据大小,weights_fname：导入参数数据文件名,lambda_func：预训练模型自带数据处理方法。

def write_gap(MODEL, image_size,weights_fname=None,lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    if weights_fname:
        print('has weights_fname')
        base_model.load_weights(weights_fname, by_name=True)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train3", image_size, shuffle=False, 
                                              batch_size=128)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=128, class_mode=None)

    train = model.predict_generator(train_generator,train_generator.samples/128+1)
    test = model.predict_generator(test_generator,test_generator.samples/128+1)
    with h5py.File("gap_f_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

In [3]:
write_gap(ResNet50, (224, 224))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [4]:
write_gap(Xception,(299, 299))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [5]:
write_gap(InceptionV3, (299, 299))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [6]:
write_gap(ResNet50, (224, 224),'w_resnet_0.h5')

has weights_fname
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [7]:
write_gap(Xception,(299, 299),"w_Xception_0.h5")

has weights_fname
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [8]:
write_gap(InceptionV3, (299, 299),"w_InceptionV3_0.h5")

has weights_fname
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
